# Setup

In [ ]:
#Setup fastai
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#Setup
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
#Other Python Libs
import random
import os
import shutil

In [28]:
# Setup Dirs

NB_DIR = Path.cwd()
MODELS = NB_DIR/'..'/'assets'/'model'
ASSETS = NB_DIR/'..'/'assets'

# Deploy the model

In [ ]:
# Import model
learn_inf = load_learner(MODELS/'mf_classifier.pkl')

In [ ]:
#Path for data to be classified
path  = ASSETS /'classify'
path.mkdir(exist_ok=True)

In [ ]:
# Header
head = widgets.HTML('<h1>Marine Fouling Classifier</h1>')
sub  = widgets.HTML('<h3>Search Images from DuckDuckGo</h3>')
# Output Widget
output = widgets.Output()
output.clear_output()

# Images
options     = ImagesCleaner()

# Search Images in DuckDuckGo
label_range = widgets.Label('Number of Images to be fetched')
txt_range   = widgets.Text()
label_query = widgets.Label('Search Query')
txt_query   = widgets.Text()
btn_search  = widgets.Button(description='Search')
btn_run     = widgets.Button(description='Classify')
ddg_img     = Image.open(ASSETS/'ddg.png').to_thumb(180,180)
ddg_logo    = widget(ddg_img)

mainBox = VBox([head,sub,label_range,txt_range,label_query,txt_query,btn_search])
header  = HBox([mainBox,ddg_logo]) 

In [ ]:
# Callback Functions

def on_run(self):
    """Manages each downloaded image associated dropdown menu and acts according to the choice made"""
    #output.clear_output()
    # Delete the selected images
    for idx in options.delete(): options.fns[idx].unlink()
        
    # Update
    fnames = get_image_files(path)
    options.set_fns(fnames)
    
    # Classify the selected images
    classifs=[]
    for idx in options.fns: 
          pred,pred_idx,probs = learn_inf.predict(idx) #use learn_inf when fixed
          lbl_pred = widgets.Label()
          lbl_pred.value = f'Prediction: {pred}'
          lbl_prob = widgets.Label()
          lbl_prob.value = f'Probability: {probs[pred_idx]:.04f}'
          classifs.append(VBox([lbl_pred,lbl_prob]))
                    

    class_labels = carousel(classifs, width='100%')
    output.clear_output()
    with output: display(options,class_labels)
    with output: display(btn_run)
        

def searchDDG(self):
    """Fetches urls of images that corresponds to the search query in DuckDuckGo
    and download them after clearing the destination folder"""
    output.clear_output()
    if path.exists and path.is_dir:
        if len(path.ls()) > 0:
            for p in path.ls(): os.unlink(p)
                
    query   = txt_query.value
    _range  = int(txt_range.value)
    
    urls = search_images_ddg(query, max_images=_range) # returns at least 100 urls
    download_images(path,urls=urls[:_range])
    fnames      = get_image_files(path)
    options.set_fns(fnames)

    with output: display(options,btn_run)

In [ ]:
# Display

btn_search.on_click(searchDDG)
btn_run.on_click(on_run)

display(header)
output

In [ ]:
# Deploy

!pip install voila
!jupyter serverextension enable --sys-prefix voila 